In [1]:
import numpy as np
import random
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('sample_data.csv')

In [3]:
data

,Zip,Age,Education,Martial Status,Occupation,Race,Sex
0,64152,39,Bachelors,Never-married,Adm-clerical,White,Male
1,61523,50,Bachelors,Married-civ-spouse,Exec-managerial,White,Male
2,95668,38,HS-grad,Divorced,Handlers-cleaners,White,Male
3,25503,53,11th,Married-civ-spouse,Handlers-cleaners,Black,Male
4,75387,28,Bachelors,Married-civ-spouse,Prof-specialty,Black,Female
...,...,...,...,...,...,...,...
58,49792,41,HS-grad,Married-civ-spouse,Adm-clerical,White,Male
59,43800,30,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male
60,54588,30,Bachelors,Married-civ-spouse,Sales,White,Male
61,21928,32,7th-8th,Married-spouse-absent,Sales,White,Male


## Task 1 - k-Anonymity and l-Diversity
Here we will work with the k-anonymity and a simplified l-diversity mechanisms. We have provided some partial code below for a few different things, and your tasks are to complete a couple of missing functions and answer some questions (See Tasks 1.1-1.4 below).

In [4]:
# A simple implementation of checking whether a list of quasi-identifiers (as tuples) is k-anonymous 
def is_k_anonymous(k, list_of_tuples):
    c = Counter(["{}{}{}".format(t[0], t[1], t[2]) for t in list_of_tuples])
    return all([c[ele] >= k for ele in c])

In [5]:
# We will need to create a list of quasi-identifiers to call the above function
qis = [(row['Zip'], row['Age'], row['Education']) for index, row in data.iterrows()]
is_k_anonymous(2, qis)

False

In [6]:
## The next three functions do different levels of generalization of the sample data, 
## and can be used to achieve k-anonymity

## Generalize zipcode by removing the last few digits
## If n is more than the number of digits, then the generalization is to 0
def generalize_zipcode(qis, n):
    return [(int(r[0]/10**n), r[1], r[2]) for r in qis]

## TASK 1.1
## You should complete the following two
def generalize_age(qis, n):
    return qis

## TASK 1.2
## For generalizing education, we will assume we are provided a mapping dictionary to map the range of
## values for that attribute to something smaller
def generalize_education(qis, mapping_dict):
    return qis

In [7]:
### Let's try a combination of generalization across the three attributes to see if we achieve k-anonymity
### for k = 3
modified_qis = generalize_zipcode(qis, 2)
modified_qis = generalize_age(modified_qis, 1)

education_mapping = {"11th": "No-College", "5th-6th": "No-College", "7th-8th": "No-College", "9th": "No-College", "HS-grad": "No-College", "Bachelors": "College", "Doctorate": "College", "Masters": "College", "Some-college": "College"}
modified_qis = generalize_education(modified_qis, education_mapping)

is_k_anonymous(2, modified_qis)

False

In [8]:
### TASK 1.3
### Did the generalizations above achieve 2-anonymity? If not, identify additional generalizations that will 
### get us to 2-anonymity.
### What additional generalization above will allow us to achieve 3-anonymity?

In [9]:
### TASK 1.4
# Implement the following simplified diversity check (let's call it x-diversity) where we require that 
# every group of tuples with the same Quasi-Identifiers has at least x different values of the sensitive attribute
# We will further assume that the last attribute in each tuple is the sensitive attribute, and the ones before
# are the QIs
def is_simplified_x_diverse(x, list_of_tuples):
    return False

# We can check that the code works with something like this -- here we treat 'race' as the sensitive/hidden attribute
qis_and_sensitive = [(row['Zip'], row['Age'], row['Education'], row['Race']) for index, row in data.iterrows()]
is_simplified_x_diverse(2, qis_and_sensitive)

False

## Task 2: Laplace Mechanism

In [10]:
## np.random has a laplace sampling function
np.random.laplace(0, 0.1)

-0.1592429982111556

In [11]:
## Task 2.1 -- write a helper function to find the new value for a given sensitivity and epsilon
def laplace_noise(v, sensitivity, epsilon):
    return v

In [12]:
## Let's try for different values of epsilon and sensitivity
## In each case, we will sample 5 times to see the kinds of values we will get in different runs
for eps in [0.1, 0.5, 1, 5, 10, 100]:
    print([laplace_noise(100, 1, eps) for _ in range(0, 5)])
for sensitivity in [1, 5, 10, 50]:
    print([laplace_noise(100, sensitivity, 0.5) for _ in range(0, 5)])

[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]


### Task 2.2 
Discuss how much noise was added for different values of epsilon and sensitivity, and whether the errors are reasoanble or not

### Task 2.3 
Generate a differentially private histogram over "Occupation"
Your task is to write the function below to take in a histogram as a dictionary and return a differentially private version of it -- you will have to figure out the right value of sensitivity to use

In [13]:
hist = dict(Counter([row['Occupation'] for index, row in data.iterrows()]))
def generate_dp_histogram(hist, epsilon):
    return hist
generate_dp_histogram(hist, 0.5)

{'Adm-clerical': 6,
 'Exec-managerial': 12,
 'Handlers-cleaners': 3,
 'Prof-specialty': 9,
 'Other-service': 5,
 'Sales': 6,
 'Craft-repair': 4,
 'Transport-moving': 3,
 'Farming-fishing': 2,
 'Machine-op-inspct': 7,
 'Tech-support': 5,
 'Protective-serv': 1}

### Task 2.4
Let's do the same for a collection of counting queries, with an example shown below.
Your task is to write the generate_dp_cq_results -- as above, you will have to figure out the right value of sensitivity to use. The function should not assume that the number of counting queries is fixed, but should use the length of the array as the number of those queries.

In [14]:
counting_queries_results = [ data[data['Occupation'] == 'Adm-clerical']['Occupation'].count(), \
                            data[data['Education'] == 'Bachelors']['Occupation'].count(), \
                            data[data['Race'] == 'Black']['Occupation'].count() ]
def generate_dp_cq_results(counting_queries_results, epsilon):
    return counting_queries_results
generate_dp_cq_results(counting_queries_results, 0.5)

[6, 16, 10]

## Task 3: Exponential Mechanism
Let's walk through an example of the exponential mechanism. We will use this to differentially output the result of query like: "Which is the most common Occupation"?

In [15]:
hist = dict(Counter([row['Occupation'] for index, row in data.iterrows()]))
## For a given occupation, the utility is defined to be the difference between max count and the count for
## that occupation
def utility_most_common(hist, occupation):
    maximum = max([hist[x] for x in hist])
    return hist[occupation] - maximum
## Stability here is the absolute difference between min and max counts
def stability_most_common(hist):
    return max([hist[x] for x in hist]) - min([hist[x] for x in hist])

In [16]:
## Your task is to write the following function that generates a differentially private 
## answer using Exponential Mechanism
## The input to this is a Counter like "hist" above -- the output should be one of the 
## words in the Counter (Occupations in this case)
## You can use the above two functions for this purpose
def generate_dp_answer_most_common(hist, epsilon):
    return 'Adm-Clerical'
generate_dp_answer_most_common(hist, 0.5)

'Adm-Clerical'

## Task 4: Simplified RAPPOR Implementation
We will simulate a simplified version of the RAPPOR protocol, using the "Occupation" attribute from the data frame above as the sensitive attribute (that the individuals want to hide).

In [17]:
## We will use an array of 0's and 1's as a bitstring
## Let's use bloom filter length, k = 10
## Let's use three hash functions: hash(s + "one"), hash(s + "two"), hash(s + "three")
BLOOM_FILTER_LENGTH = 10
PROB_PERM_RESPONSE = 0.75
PROB_INSTA_RESPONSE = 0.75

## Construct the bloom filter for a string
def bloom_filter(s):
    ret = [0 for _ in range(0, BLOOM_FILTER_LENGTH)]
    ret[hash(s + "one")%BLOOM_FILTER_LENGTH] = 1
    ret[hash(s + "two")%BLOOM_FILTER_LENGTH] = 1    
    ret[hash(s + "three")%BLOOM_FILTER_LENGTH] = 1
    return ret

def randomized_response(act_value, prob):
    if random.random() < prob:
        return act_value
    return 1 - act_value   ## return 0 if act_value = 1 and vice versa

### Permanent responses are different for each user who is contributing data
perm_responses_by_user = [dict() for _ in range(0, 100)]

def perm_response(s, userindex):
    if not s in perm_responses_by_user[userindex]:
        bf = bloom_filter(s)
        for index in range(0, len(bf)):
            bf[index] = randomized_response(bf[index], PROB_PERM_RESPONSE)
        perm_responses_by_user[userindex][s] = bf
    return perm_responses_by_user[userindex][s]

## Obtain the instantenous response for a string for a user, by first getting the perm_response and then 
## randomizing it
def insta_response(s, userindex):
    bf_ir = [x for x in perm_response(s, userindex)]
    for index in range(0, len(bf_ir)):
        bf_ir[index] = randomized_response(bf_ir[index], PROB_INSTA_RESPONSE)
    return bf_ir

In [18]:
## We can confirm that perm_response for a given string, userindex combination is always the same
print("Original {}".format(bloom_filter("Adm-clerical")))
print("Permanent responses")
for i in range(0, 5):
    print(perm_response("Adm-clerical", 0))
## But insta_response is not
print("Instateneous responses")
for i in range(0, 5):
    print(insta_response("Adm-clerical", 0))

Original [1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
Permanent responses
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
Instateneous responses
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 1, 0, 0, 1]


In [19]:
### Let's simulate a full run of a simplified version of RAPPOR
### We will assume we have 10 users, who each are producing a collection of strings
### For each of their strings, each user will return an instantenous response

### Let's create the data
### by picking a random subset from the data frame above as the data for each user
### For repeatability, we will set the seed here
random.seed(100)
user_data = []
for i in range(0, 10):
    user_data.append([])
    for index, row in data.iterrows():
        if random.random() < 0.6:
            user_data[i].append(row['Occupation'])

In [20]:
### Our end goal here is to collect a histogram over all the strings in the user_data
### in a privacy-preserving manner
### Here is the correct answer, i.e., "Ground Truth" so to say
ground_truth = Counter([x for x in user_data[i] for i in range(0, 10)])
ground_truth

Counter({'Adm-clerical': 50,
         'Exec-managerial': 60,
         'Handlers-cleaners': 20,
         'Prof-specialty': 60,
         'Other-service': 10,
         'Sales': 40,
         'Transport-moving': 10,
         'Farming-fishing': 10,
         'Tech-support': 40,
         'Craft-repair': 10,
         'Protective-serv': 10,
         'Machine-op-inspct': 50})

In [21]:
### The "collector" (who is untrusted in this case) only gets to see the "insta_responses"
collected_data = []
for i in range(0, 10):
    for x in user_data[i]:
        collected_data.append(insta_response(x, i))
collected_data[0:10]

[[1, 0, 0, 1, 1, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 1, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 1, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

### Task 4.1
You have to implement the "recovery" portion using a simpler approach than what RAPPOR did
- Step 1: Sum up all the collected bloom filters to get a single vector of size 10
- Step 2: Scale the observed numbers based on the probabilities we are using 
- Step 3: Find the best vector over input strings that matches the output we see using a regression package

We have done Step 3 below, but you have to implement Steps 1 and 2

In [22]:

def find_scaling_factor(prob1, prob2):
    return 1
def find_sum_vector(collected_data):
    return [0 for _ in range(0, 10)]

sum_vector = find_sum_vector(collected_data)
scaling_factor = find_scaling_factor(0.75, 0.75) ### These are the two probabilities we use above
scaled_sum_vector = [x * scaling_factor for x in sum_vector]

### We can now construct design matrix X
all_occupations = set([row['Occupation'] for index, row in data.iterrows()])
X = [bloom_filter(x) for x in all_occupations]
estimated_counts = [0 for _ in range(0, 10)]

### We are looking for the estimated_counts vector that best matches the scaled_sum_vector that we saw
### In other words, we want X * estimated_counts to be as close to scaled_sum_vector as possible

In [23]:
### We can use the linear algebra package least squared algorithm to find the best fit
np.linalg.lstsq(np.array(X).T, np.array(scaled_sum_vector))[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Task 4.2
Discuss how well the answers matched the above answers. Regenerate the "collected answers" (so that you would be using different randomization for insta_responses) and discuss what impact it had on the estimated answers.

You can write your answer in the next empty cell.